In [2]:
import torch
import librosa
import numpy as np
import torch.nn as nn
from PIL import Image
import librosa.display
import os
import torchvision
from torchvision import transforms
from torch.autograd import Variable

In [7]:
output_dim = [5,16,45,70]
kernel_size = [5,3,3,3]
pooling_size = [2,2,2,2]
stride_size = [1,1,1,1]
paddind_size = [1,0,0,0]
dropout_size = [0.1,0.1,0.1,0.1]
# 定义 Convolution Network 模型
class Cnn(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Cnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, output_dim[0], kernel_size[0], stride=stride_size[0], padding=paddind_size[0]),
            nn.MaxPool2d(pooling_size[0], pooling_size[0]),
            nn.ReLU(True),
            nn.Dropout2d(p=dropout_size[0]),

            nn.Conv2d(output_dim[0], output_dim[1], kernel_size[1], stride=stride_size[1], padding=stride_size[1]),
            nn.MaxPool2d(pooling_size[1], pooling_size[1]),
            nn.ReLU(True),
            nn.Dropout2d(p=dropout_size[1]),

            nn.Conv2d(output_dim[1],output_dim[2],kernel_size[2], stride=stride_size[2], padding=stride_size[2]),
            nn.MaxPool2d(pooling_size[2], pooling_size[2]),
            nn.ReLU(True),
            nn.Dropout2d(p=dropout_size[2]),

            nn.Conv2d(output_dim[2],output_dim[3],kernel_size[3], stride=stride_size[3], padding=stride_size[3]),
            nn.MaxPool2d(pooling_size[3], pooling_size[3]),
            nn.ReLU(True),
            nn.Dropout2d(p=dropout_size[3]))

        '''
            nn.Conv2d(30, 50, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2))
    
            nn.Conv2d(50, 100, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2))
        '''
        self.fc = nn.Sequential(
            nn.Linear(10080, 120), nn.Linear(120, 60), nn.Linear(60, 30),nn.Linear(30,n_class))
            #nn.Linear(14520, 120), nn.Linear(120, 84), nn.Linear(84, n_class))

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])

custom_trans = transforms.Compose([\
            transforms.RandomResizedCrop(224),\
            transforms.RandomHorizontalFlip(),\
            transforms.ToTensor(),\
            normalize])

transform2 = transforms.Compose([
    transforms.ToTensor(),
    normalize])

In [8]:
model = Cnn(3,2)

In [27]:
model.load_state_dict(torch.load("/home/qinjianbo/PycharmProjects/yinnao_torch/cnn.pth"))
model.eval()

Cnn(
  (conv): Sequential(
    (0): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (2): ReLU(inplace)
    (3): Dropout2d(p=0.1)
    (4): Conv2d(5, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (6): ReLU(inplace)
    (7): Dropout2d(p=0.1)
    (8): Conv2d(16, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (10): ReLU(inplace)
    (11): Dropout2d(p=0.1)
    (12): Conv2d(45, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (14): ReLU(inplace)
    (15): Dropout2d(p=0.1)
  )
  (fc): Sequential(
    (0): Linear(in_features=10080, out_features=120, bias=True)
    (1): Linear(in_features=120, o

In [52]:
#img = plt.imread("/home/qinjianbo/DATA/audio_sample/samples/胡茄十八拍/specs/胡茄十八拍_11.png")
#img = plt.imread("/home/qinjianbo/DATA/audio_sample/train/1/乌夜啼_1094.png")
#img = Image.open("/home/qinjianbo/DATA/audio_sample/train/1/乌夜啼_1094.png")
file = "/home/qinjianbo/DATA/audio_sample/test/1/胡茄十八拍_2589.png"
file = "/home/qinjianbo/DATA/audio_sample/samples/胡茄十八拍/specs/胡茄十八拍_973.png"
#file = "/home/qinjianbo/DATA/audio_sample/train/1/乌夜啼_1094.png"
file = "/home/qinjianbo/DATA/audio_sample/test/1/乌夜啼_1564.png"
#file = "/home/qinjianbo/DATA/audio_sample/test/1/秋鸿_1883.png"
img = Image.open(file)
img = np.array(img)
img = transform2(img)
img = Variable(img.view(1,3,200,200))
model(img)

Variable containing:
-5.5363  1.7269
[torch.FloatTensor of size 1x2]

In [22]:
img.shape

torch.Size([1, 3, 200, 200])

In [46]:
model.training

False